In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Using Solver Options

All solvers (both nonlinear and linear) have a number of options that you access via the `options` attribute that control its behavior:

In [ ]:
import openmdao.api as om
om.show_options_table("openmdao.solvers.solver.Solver")

## Iteration Limit and Convergence Tolerances

Here is how you would change the iteration limit and convergence tolerances for [NonlinearBlockGS](../../building_blocks/solvers/nonlinear_block_gs).

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src46", get_code("openmdao.test_suite.components.sellar.SellarDis1withDerivatives"), display=False)

:::{Admonition} `SellarDis1withDerivatives` class definition 
:class: dropdown

{glue:}`code_src46`
:::

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src47", get_code("openmdao.test_suite.components.sellar.SellarDis2withDerivatives"), display=False)

:::{Admonition} `SellarDis2withDerivatives` class definition 
:class: dropdown

{glue:}`code_src47`
:::

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src48", get_code("openmdao.test_suite.components.sellar_feature.SellarDerivatives"), display=False)

:::{Admonition} `SellarDerivatives` class definition 
:class: dropdown

{glue:}`code_src48`
:::

In [ ]:
import numpy as np

import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem(model=SellarDerivatives())

nlbgs = prob.model.nonlinear_solver = om.NonlinearBlockGS()

nlbgs.options['maxiter'] = 20
nlbgs.options['atol'] = 1e-6
nlbgs.options['rtol'] = 1e-6

prob.setup()
prob.set_val('x', 1.)
prob.set_val('z', np.array([5.0, 2.0]))
prob.run_model()

print(prob.get_val('y1'))
print(prob.get_val('y2'))

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(prob.get_val('y1'), 25.58830273, .00001)
assert_near_equal(prob.get_val('y2'), 12.05848819, .00001)

## Displaying Solver Convergence Info

Solvers can all print out some information about their convergence history. If you want to control that printing behavior you can use the `iprint` option in the solver.


**iprint = -1**: Print nothing

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem()
prob.model = SellarDerivatives()

prob.setup()

newton = prob.model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
scipy = prob.model.linear_solver = om.ScipyKrylov()

newton.options['maxiter'] = 2

# use a real bad initial guess
prob.set_val('y1', 10000)
prob.set_val('y2', -26)

newton.options['iprint'] = -1
scipy.options['iprint'] = -1
prob.run_model()


**iprint = 0**: Print only errors or convergence failures.

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem()
prob.model = SellarDerivatives()

prob.setup()

newton = prob.model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
scipy = prob.model.linear_solver = om.ScipyKrylov()

newton.options['maxiter'] = 1

prob.set_val('y1', 10000)
prob.set_val('y2', -26)

newton.options['iprint'] = 0
scipy.options['iprint'] = 0

prob.run_model()



**iprint = 1**: Print a convergence summary, as well as errors and convergence failures.

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem(model=SellarDerivatives())

prob.setup()

newton = prob.model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
scipy = prob.model.linear_solver = om.ScipyKrylov()

newton.options['maxiter'] = 20

prob.set_val('y1', 10000)
prob.set_val('y2', -26)

newton.options['iprint'] = 1
scipy.options['iprint'] = 0
prob.run_model()


**iprint = 2**: Print the residual for every solver iteration.

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem(model=SellarDerivatives())

prob.setup()

newton = prob.model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
scipy = prob.model.linear_solver = om.ScipyKrylov()

newton.options['maxiter'] = 20

prob.set_val('y1', 10000)
prob.set_val('y2', -26)

newton.options['iprint'] = 2
scipy.options['iprint'] = 1
prob.run_model()

(solver-options-set_solver-print)=
## Controlling Solver Output in Large Models

When you have a large model with multiple solvers, it is easier to use a shortcut method that
recurses over the entire model. The `set_solver_print` method on `problem` can be used to
set the iprint to one of four specific values for all solvers in the model while specifically
controlling depth (how many systems deep) and the solver type (linear, nonlinear, or both.)

To print everything, just call `set_solver_print` with a level of "2".

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src49", get_code("openmdao.test_suite.components.double_sellar.SubSellar"), display=False)

:::{Admonition} `SubSellar` class definition 
:class: dropdown

{glue:}`code_src49`
:::

In [ ]:
import numpy as np
from openmdao.test_suite.components.double_sellar import SubSellar

prob = om.Problem()
model = prob.model

sub1 = model.add_subsystem('sub1', om.Group(), promotes_inputs=['z'])
sub2 = sub1.add_subsystem('sub2', om.Group(), promotes_inputs=['z'])
g1 = sub2.add_subsystem('g1', SubSellar(), promotes_inputs=['z'])
g2 = model.add_subsystem('g2', SubSellar())

model.connect('sub1.sub2.g1.y2', 'g2.x')
model.connect('g2.y2', 'sub1.sub2.g1.x')

model.nonlinear_solver = om.NewtonSolver()
model.linear_solver = om.ScipyKrylov()
model.nonlinear_solver.options['solve_subsystems'] = True
model.nonlinear_solver.options['max_sub_solves'] = 0

g1.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
g1.linear_solver = om.LinearBlockGS()

g2.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
g2.linear_solver = om.ScipyKrylov()
g2.linear_solver.precon = om.LinearBlockGS()
g2.linear_solver.precon.options['maxiter'] = 2

prob.set_solver_print(level=2)

prob.setup()
prob.set_val('z', np.array([5.0, 2.0]))
prob.run_model()

To print everything for nonlinear solvers, and nothing for the linear solvers, first turn everything
on, as shown above, and then call `set_solver_print` again to set a level of "-1" on just the linear solvers (using the `type_` argument), so that we suppress everything, including the messages when the linear block Gauss-Seidel solver hits the maximum iteration limit. You can call the `set_solver_print` method multiple times to stack different solver print types in your model.

In [ ]:
from openmdao.test_suite.components.double_sellar import SubSellar

prob = om.Problem()
model = prob.model

sub1 = model.add_subsystem('sub1', om.Group(), promotes_inputs=['z'])
sub2 = sub1.add_subsystem('sub2', om.Group(), promotes_inputs=['z'])
g1 = sub2.add_subsystem('g1', SubSellar(), promotes_inputs=['z'])
g2 = model.add_subsystem('g2', SubSellar())

model.connect('sub1.sub2.g1.y2', 'g2.x')
model.connect('g2.y2', 'sub1.sub2.g1.x')

model.nonlinear_solver = om.NewtonSolver()
model.linear_solver = om.ScipyKrylov()
model.nonlinear_solver.options['solve_subsystems'] = True
model.nonlinear_solver.options['max_sub_solves'] = 0

g1.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
g1.linear_solver = om.LinearBlockGS()

g2.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
g2.linear_solver = om.ScipyKrylov()
g2.linear_solver.precon = om.LinearBlockGS()
g2.linear_solver.precon.options['maxiter'] = 2

prob.set_solver_print(level=2)
prob.set_solver_print(level=-1, type_='LN')

prob.setup()
prob.set_val('z', np.array([5.0, 2.0]))
prob.run_model()

If we just want to print solver output for the first level of this multi-level model, we first turn
off all printing, and then set a print level of "2" with a `depth` argument of "2" so that we only print the
top solver and the solver in 'g2', but not the solver in 'sub1.sub2.g1'.

In [ ]:
from openmdao.test_suite.components.double_sellar import SubSellar

prob = om.Problem()
model = prob.model

sub1 = model.add_subsystem('sub1', om.Group(), promotes_inputs=['z'])
sub2 = sub1.add_subsystem('sub2', om.Group(), promotes_inputs=['z'])
g1 = sub2.add_subsystem('g1', SubSellar(), promotes_inputs=['z'])
g2 = model.add_subsystem('g2', SubSellar())

model.connect('sub1.sub2.g1.y2', 'g2.x')
model.connect('g2.y2', 'sub1.sub2.g1.x')

model.nonlinear_solver = om.NewtonSolver()
model.linear_solver = om.ScipyKrylov()
model.nonlinear_solver.options['solve_subsystems'] = True
model.nonlinear_solver.options['max_sub_solves'] = 0

g1.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
g1.linear_solver = om.LinearBlockGS()

g2.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
g2.linear_solver = om.ScipyKrylov()
g2.linear_solver.precon = om.LinearBlockGS()
g2.linear_solver.precon.options['maxiter'] = 2

prob.set_solver_print(level=0)
prob.set_solver_print(level=2, depth=2)

prob.setup()
prob.set_val('z', np.array([5.0, 2.0]))
prob.run_model()

The `set_solver_print` method can also be called on Systems. For instance, if we want to print detailed output from group 'g2' down, we can first call `set_solver_print` on the problem or the top level model with a level of "-1", and then call it on group 'g2' with a level of "2".

In [ ]:
from openmdao.test_suite.components.double_sellar import SubSellar

prob = om.Problem()
model = prob.model

model.add_subsystem('pz', om.IndepVarComp('z', np.array([5.0, 2.0])))

sub1 = model.add_subsystem('sub1', om.Group())
sub2 = sub1.add_subsystem('sub2', om.Group())
g1 = sub2.add_subsystem('g1', SubSellar())
g2 = model.add_subsystem('g2', SubSellar())

model.connect('sub1.sub2.g1.y2', 'g2.x')
model.connect('g2.y2', 'sub1.sub2.g1.x')

model.nonlinear_solver = om.NewtonSolver()
model.linear_solver = om.ScipyKrylov()
model.nonlinear_solver.options['solve_subsystems'] = True
model.nonlinear_solver.options['max_sub_solves'] = 0

g1.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
g1.linear_solver = om.LinearBlockGS()

g2.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
g2.linear_solver = om.ScipyKrylov()
g2.linear_solver.precon = om.LinearBlockGS()
g2.linear_solver.precon.options['maxiter'] = 2

prob.set_solver_print(level=-1, type_='all')
g2.set_solver_print(level=2, type_='NL')

prob.setup()
prob.run_model()